In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")

In [ ]:
df.head()

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
df['datetime']

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(30)

In [ ]:
df['date'] =df['datetime'].dt.date
df['time'] = df['datetime'].dt.time

In [ ]:
df['date'][2]

In [ ]:
df['year'] = df['datetime'].dt.year.astype(float)
df['month'] = df['datetime'].dt.month.astype(float)
df['day'] = df['datetime'].dt.day.astype(float)
df['day_of_week'] = pd.to_datetime(df['datetime']).dt.dayofweek.astype(float)
df['hour']=df['datetime'].dt.hour.astype(float)

In [ ]:
type(df['year'][3])

In [ ]:
type(df['hour'][3])

In [ ]:
type(df['day_of_week'][3])

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['datetime', 'holiday', 'temp', 'casual','registered', 'date', 'time','day'], axis = 1)

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('darkgrid')
chart = sns.barplot(x = df['hour'], y = df['count'])
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
df.groupby('hour')['count'].sum().plot.bar()

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('darkgrid')
chart = sns.barplot(x = df['season'], y = df['count'], hue = df['workingday'])
chart.set_xticklabels(chart.get_xticklabels(),  horizontalalignment='right')

In [ ]:
df['year']

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('darkgrid')
chart = sns.barplot(x = df['year'], y = df['count'])
chart.set_xticklabels(chart.get_xticklabels(),  horizontalalignment='right')

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('darkgrid')
chart = sns.barplot(x = df['month'], y = df['count'])
chart.set_xticklabels(chart.get_xticklabels(),  horizontalalignment='right')

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style('darkgrid')
chart = sns.barplot(x = df['day_of_week'], y = df['count'])
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
plt.figure(figsize=(16,8))
sns.set_style('darkgrid')
chart = sns.barplot(x = df['weather'], y = df['count'], hue= df['hour'])
chart.set_xticklabels(chart.get_xticklabels(), rotation=0, horizontalalignment='right')

In [ ]:
#for working days:
#rush_hour = {7: 'rush_mor', 8:'rush_mor', 9:'rush_mor', 17:'rush_eve', 18:'rush_eve', 19:'rush_eve', 11: 'vac_day', 12: 'vac_day', 13:'', 14:'', 15:'', 16:''}

In [ ]:
#df['rush_hour']= df['time'].map(rush_hour)

In [ ]:
df.shape


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
corrMatrix =pd.DataFrame(X_train.corr())
print(corrMatrix)

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
set(df['windspeed'])

In [ ]:
set(df['atemp'])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')

x = df['humidity']
y = df['windspeed']
z = df['atemp']
c = df['count']

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
plt.xlabel('humidity', fontsize=22, labelpad=20)
plt.ylabel('windspeed', fontsize=22, labelpad=20)
#plt.zlabel('atemp', fontsize=22, labelpad=25)
ax.view_init(45,60)
fig.colorbar(img)
plt.show()

In [ ]:
y_train = X_train['count']
X_train = X_train.drop(columns = ['count'], axis=1)

In [ ]:
y_test = X_test['count']
X_test = X_test.drop(columns = ['count'], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#scaler = MinMaxScaler()
#X_train['atemp'] = pd.Series(scaler.fit_transform([X_train['atemp']]))

cols_to_norm = ['atemp','windspeed', 'humidity']
X_train[cols_to_norm] = MinMaxScaler().fit_transform(X_train[cols_to_norm])
X_test[cols_to_norm] = MinMaxScaler().fit_transform(X_test[cols_to_norm])

#X_train_scaled = preprocessing.normalize(X_train)
#X_test_scaled = preprocessing.normalize(X_test)

In [ ]:
#X_train_scaled

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error

In [ ]:
linearRegressor = LinearRegression()
linearRegressor.fit(X_train, y_train)
y_predicted = linearRegressor.predict(X_test)
y_predicted_rmse = y_predicted.clip(0)


In [ ]:
y_predicted_rmse.min()




In [ ]:
mse = mean_squared_error(y_test, y_predicted)
r = r2_score(y_test, y_predicted)
mae = mean_absolute_error(y_test,y_predicted)
msle=mean_squared_log_error(y_predicted_rmse,y_test)
rmsle=np.sqrt(msle)

print('RMLSE for the data:',rmsle)
print("Mean Squared Error:",mse)
print("R score:",r)
print("Mean Absolute Error:",mae)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(y_test)
plt.plot(y_predicted)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size=0.25, random_state=42)
y_train = X_train['count']
X_train = X_train.drop(columns = ['count'], axis=1)
y_test = X_test['count']
X_test = X_test.drop(columns = ['count'], axis=1)

In [ ]:
from statsmodels.api import OLS, add_constant
ols = OLS(y_train, add_constant(X_train))
ols_results = ols.fit()
ols_results.summary()

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
rf = RandomForestRegressor(max_depth=5, random_state=0)
rfe = RFE(rf, n_features_to_select=3)
rfe.fit(X_train, y_train)

rfe_df = pd.DataFrame({'support': rfe.support_, 'ranking': rfe.ranking_, 'feature': X_train.columns})
rfe_df.sort_values('ranking')

In [ ]:
# Finding best parameters for RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf_params = {‘n_estimators’:np.arange(25,150,25),’max_depth’:np.arange(1,11,2),’min_samples_leaf’:np.arange(2,15,3)}

from sklearn.model_selection import GridSearchCV
gs_rf = GridSearchCV(rf,rf_params,cv=3)
gs_rf.fit(x_train,y_train)
b = gs_rf.best_params_




In [ ]:
# Fitting the model with best params
RF = RandomForestRegressor(n_estimators=b['n_estimators'],max_depth=b['max_depth'],min_samples_leaf=b['min_samples_leaf'],random_state=0)
model = RF.fit(x_train,y_train)
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_log_error
msle=mean_squared_log_error(y_pred,y_test)
rmsle=np.sqrt(msle)
print('RMLSE for the data:',rmsle) # For random forest